# A brief introduction to the OpenAI API

Before starting to create the chatbot, I think it is interesting to explain a couple of points:

- The roles within a conversation with OpenAI.
- How is the conversations’ memory preserved?

If you prefer to start creating the chatbot, just move to the section: Creating the Chatbot with OpenAI and GPT.

## The roles in OpenAI messages

One of the lesser-known features of language models such as GPT 3.5 is that the conversation occurs between several roles. We can identify the user and the assistant, but there is a third role called system, which allows us to better configure how the model should behave.

When we use tools like ChatGPT, we always assume the role of the user, but the API lets us choose which Role we want to send to the model, for each sentence.

To send text containing our part of the dialog to the model, we must use the `ChatCompletion.create` function, indicating, at least, the model to use and a list of messages.

Each message in the list contains a role and the text we want to send to the model.

Here is an example of the list of messages that can be sent using the three available roles.


In [5]:
 messages=[
        {"role": "system", "content": "You are an OrderBot in a fastfood restaurant."},
        {"role": "user", "content": "I have only 10 dollars, what can I order?"},
        {"role": "assistant", "content": "We have the fast menu for 7 dollars."},
        {"role": "user", "content": "Perfect! Give me one! "}
    ]

Let’s take a closer look at the three existing roles:

- **System:** We can tell the model how we want it to behave and tell it how its personality and type of response should be. Somehow, it allows us to configure the basic operation of the model. OpenAI says that this role will become more important in the next models, even though now its importance is relatively small in GPT 3.5.
- **User:** These are the phrases that come from the user.
- **Assistant:** These are the responses returned by the model. With the API, we can send responses that say they came from the model, even if they came from somewhere else.


# Vertical Chat
A sample how to build a chat for small businees using:

* GPT 35
* Panel
* OpenAI


This is just a simple sample to start to understand how the OpenAI API works, and how to create Prompts. It Is really far from beign a complete solution.
We are going to introduce some interesting points:

* The roles in a conversation.
* How is the conversations’ memory preserved?

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.


# Creating the Chatbot with OpenAI and GPT

The first thing we have to consider is that we are going to need an OpenAI payment account to use their service and that we will have to report a valid credit card. But let’s not worry, I’ve been using it a lot for development and testing, and I can assure you that the cost is negligible.

Doing all the tests for this article, I think they cost me €0.07. We could only be surprised if we upload something to production that becomes a HIT. Even so, we can establish the monthly consumption limit that we want.

The first thing, as always, is to know if we have the necessary libraries installed. In case we work on Google Colab, I think we only have to install two, OpenAI and panel.


In [2]:
!pip install openai==1.1.1
!pip install panel

In [2]:
import openai

openai.api_key="your-api-key"

## Formatting the answer with Few Shot Samples.
To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:

Zero-Shot.
One-Shot.
Few-Shots.
With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



My key is stored in a file where I keep the keys. But if you like, you can inform it directly in the notebook, or save the key in a file, with a .py extension.

In any case, make sure that nobody can ever know the value of the Key; otherwise, they could make calls to the OpenAI API that you would end up paying for.

Now we are going to define two functions, which will be the ones that will contain the logic of maintaining the memory of the conversation.


In [6]:
def continue_conversation(messages, temperature=0):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temperature,
    )
    #print(str(response.choices[0].message["content"]))
    return response.choices[0].message.content

This function is very simple, it just makes a call to the OpenAI API that allows you to have a conversation.

Inside the function, we are calling OpenAI with the following parameters:

- `model`: the model we want to use.
- `messages`: the messages part of the conversation.
- `temperature`: It is a numerical value between 0 and 1, which indicates how imaginative the model can be when generating the response. The smaller the value, the less original the model’s response will be.

As you know, a language generation model does not always give the same answers to the same inputs. The lower the value of temperature, the more similar the result will be for the same inputs, even repeating itself in many cases.


In [ ]:
def add_prompts_conversation(_):
    #Get the value introduced by the user
    prompt = client_prompt.value_input
    client_prompt.value = ''

    #Append to the context the User prompt.
    context.append({'role':'user', 'content':f"{prompt}"})

    #Get the response.
    response = continue_conversation(context)

    #Add the response to the context.
    context.append({'role':'assistant', 'content':f"{response}"})

    #Update the panels to show the conversation.
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600)))

    return pn.Column(*panels)

This function is responsible for collecting user input, incorporating it into the context or conversation, calling the model, and incorporating its response into the conversation. That is, it is responsible for managing the memory! It is as simple as adding phrases with the correct format to a list, where each sentence is formed by the role and the phrase.

Now is the time for the prompt!

This is an LLM model. We are not going to program, we are going to try to make it behave as we want by giving it some instructions. At the same time, we must also provide it with enough information so that it can do its job properly informed.


In [3]:
context = [ {'role':'system', 'content':"""
Act as an OrderBot, you work collecting orders in a delivery only fast food restaurant called
My Dear Frankfurt. \
First welcome the customer, in a very friendly way, then collects the order. \
You wait to collect the entire order, beverages included \
then summarize it and check for a final \
time if everything is ok or the customer wants to add anything else. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very friendly style. \
The menu includes \
burger  12.95, 10.00, 7.00 \
frankfurt   10.95, 9.25, 6.50 \
sandwich   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
martra sausage 3.00 \
canadian bacon 3.50 \
romesco sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
vichy catalan 5.00 \
"""} ]

#Creating the panel.
pn.extension()

panels = []

client_prompt = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="talk")

interactive_conversation = pn.bind(add_prompts_conversation, button_conversation)

dashboard = pn.Column(
    client_prompt,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True),
)

dashboard